<a href="https://colab.research.google.com/github/MeghanaVajrapu/Air-Quality-Prediction-System/blob/main/air_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/AirQualityUCI.xlsx')

In [ ]:
df.to_csv('/content/drive/MyDrive/AirQualityUCI.csv', index=False)


In [ ]:
df.head()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,2004-03-10,18:00:00,2.6,1360.00,150,11.881723,1045.50,166.0,1056.25,113.0,1692.00,1267.50,13.60,48.875001,0.757754
1,2004-03-10,19:00:00,2.0,1292.25,112,9.397165,954.75,103.0,1173.75,92.0,1558.75,972.25,13.30,47.700000,0.725487
2,2004-03-10,20:00:00,2.2,1402.00,88,8.997817,939.25,131.0,1140.00,114.0,1554.50,1074.00,11.90,53.975000,0.750239
3,2004-03-10,21:00:00,2.2,1375.50,80,9.228796,948.25,172.0,1092.00,122.0,1583.75,1203.25,11.00,60.000000,0.786713
4,2004-03-10,22:00:00,1.6,1272.25,51,6.518224,835.50,131.0,1205.00,116.0,1490.00,1110.00,11.15,59.575001,0.788794


In [ ]:
# Drop unnecessary columns and rename columns
columns_to_drop = ['Unnamed: 15', 'Unnamed: 16']
existing_columns = [col for col in columns_to_drop if col in df.columns]
if existing_columns:
    df = df.drop(existing_columns, axis=1)
df = df.rename(columns={'Date_Time': 'Date', 'CO(GT)': 'CO', 'C6H6(GT)': 'Benzene', 'NOx(GT)': 'NOx', 'NO2(GT)': 'NO2'})

In [ ]:
# Drop rows with missing values
df = df.dropna()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9357 entries, 0 to 9356
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           9357 non-null   datetime64[ns]
 1   Time           9357 non-null   object        
 2   CO             9357 non-null   float64       
 3   PT08.S1(CO)    9357 non-null   float64       
 4   NMHC(GT)       9357 non-null   int64         
 5   Benzene        9357 non-null   float64       
 6   PT08.S2(NMHC)  9357 non-null   float64       
 7   NOx            9357 non-null   float64       
 8   PT08.S3(NOx)   9357 non-null   float64       
 9   NO2            9357 non-null   float64       
 10  PT08.S4(NO2)   9357 non-null   float64       
 11  PT08.S5(O3)    9357 non-null   float64       
 12  T              9357 non-null   float64       
 13  RH             9357 non-null   float64       
 14  AH             9357 non-null   float64       
dtypes: datetime64[ns](1),

In [ ]:
df.head()

,Date,Time,CO,PT08.S1(CO),NMHC(GT),Benzene,PT08.S2(NMHC),NOx,PT08.S3(NOx),NO2,PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,2004-03-10,18:00:00,2.6,1360.00,150,11.881723,1045.50,166.0,1056.25,113.0,1692.00,1267.50,13.60,48.875001,0.757754
1,2004-03-10,19:00:00,2.0,1292.25,112,9.397165,954.75,103.0,1173.75,92.0,1558.75,972.25,13.30,47.700000,0.725487
2,2004-03-10,20:00:00,2.2,1402.00,88,8.997817,939.25,131.0,1140.00,114.0,1554.50,1074.00,11.90,53.975000,0.750239
3,2004-03-10,21:00:00,2.2,1375.50,80,9.228796,948.25,172.0,1092.00,122.0,1583.75,1203.25,11.00,60.000000,0.786713
4,2004-03-10,22:00:00,1.6,1272.25,51,6.518224,835.50,131.0,1205.00,116.0,1490.00,1110.00,11.15,59.575001,0.788794


In [ ]:
# Convert data types and remove negative values
cols = ['CO', 'PT08.S1(CO)', 'NMHC(GT)', 'Benzene', 'PT08.S2(NMHC)', 'NOx', 'PT08.S3(NOx)', 'NO2', 'PT08.S4(NO2)', 'PT08.S5(O3)', 'T', 'RH', 'AH']
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
df = df[(df[cols] >= 0).all(axis=1)]

In [ ]:
# EDA
print(df.head())
print(df.describe())

        Date      Time   CO  PT08.S1(CO)  NMHC(GT)    Benzene  PT08.S2(NMHC)  \
0 2004-03-10  18:00:00  2.6      1360.00       150  11.881723        1045.50   
1 2004-03-10  19:00:00  2.0      1292.25       112   9.397165         954.75   
2 2004-03-10  20:00:00  2.2      1402.00        88   8.997817         939.25   
3 2004-03-10  21:00:00  2.2      1375.50        80   9.228796         948.25   
4 2004-03-10  22:00:00  1.6      1272.25        51   6.518224         835.50   

     NOx  PT08.S3(NOx)    NO2  PT08.S4(NO2)  PT08.S5(O3)      T         RH  \
0  166.0       1056.25  113.0       1692.00      1267.50  13.60  48.875001   
1  103.0       1173.75   92.0       1558.75       972.25  13.30  47.700000   
2  131.0       1140.00  114.0       1554.50      1074.00  11.90  53.975000   
3  172.0       1092.00  122.0       1583.75      1203.25  11.00  60.000000   
4  131.0       1205.00  116.0       1490.00      1110.00  11.15  59.575001   

         AH  
0  0.757754  
1  0.725487  
2  0.750

In [ ]:
# Prepare data for modeling
X = df[['CO', 'Benzene', 'NOx', 'NO2', 'T', 'RH', 'AH']]
y = df['PT08.S1(CO)'].values.ravel()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train linear regression model
reg = LinearRegression().fit(X_train, y_train)



In [ ]:
# Evaluate model
y_pred = reg.predict(X_test)
print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))
print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred))



Mean squared error: 5123.80
Coefficient of determination: 0.92


In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Train and evaluate Decision Tree Regressor
dtr = DecisionTreeRegressor(random_state=42)
dtr.fit(X_train, y_train)
y_pred_dtr = dtr.predict(X_test)
print('Decision Tree Regressor - Mean squared error: %.2f' % mean_squared_error(y_test, y_pred_dtr))
print('Decision Tree Regressor - Coefficient of determination: %.2f' % r2_score(y_test, y_pred_dtr))

Decision Tree Regressor - Mean squared error: 8740.70
Decision Tree Regressor - Coefficient of determination: 0.86


In [ ]:
# Train and evaluate Random Forest Regressor
rfr = RandomForestRegressor(random_state=42)
rfr.fit(X_train, y_train)
y_pred_rfr = rfr.predict(X_test)
print('Random Forest Regressor - Mean squared error: %.2f' % mean_squared_error(y_test, y_pred_rfr))
print('Random Forest Regressor - Coefficient of determination: %.2f' % r2_score(y_test, y_pred_rfr))

Random Forest Regressor - Mean squared error: 5401.50
Random Forest Regressor - Coefficient of determination: 0.92


In [ ]:
import joblib

print("joblib imported successfully.")

joblib imported successfully.


In [ ]:
#save the model (rfr) to a file named 'model.joblib'

joblib.dump(rfr, 'model.joblib')
print("Model saved as model.joblib")

Model saved as model.joblib
